In [104]:
import numpy as np
import requests
import json
import datetime
import time
import pandas as pd
from pandas.io.json import json_normalize
from datetime import date
from datetime import timedelta
from bs4 import BeautifulSoup

# Helper functions and Classes

In [120]:
def extract_subtable(df, columns, label="Plane"):
    """Extract a sub-table which have columns labelled with numerical indexes.
    
    Parameters
    ----------
    df : pandas dataframe
        Input dataframe holding all the columns needed to extract.
    columns : list of str
        List of columns labels to read from.
    label : str, optional
        Label for the sub-table index.
        
    Returns
    -------
    subdf : pandas dataframe
        Output dataframe with MultiIndex for customer and label indices.
    """
    subdf = df[columns].copy(deep=True)
    
#     columns_noindex = subdf.columns.str.extract(r'([a-zA-Z\-_]*)([0-9]+)([a-zA-Z\-_]*)')
#     columns_noindex[0] = columns_noindex[0] + columns_noindex[2]
#     columns_noindex[1] = columns_noindex[1].astype(int)
#     columns_noindex.drop(2, axis=1, inplace=True)
    
    subdf.columns = pd.MultiIndex.from_frame(columns, names=["", f'{label}Idx'])
    subdf = subdf.stack()
    return subdf

In [106]:
username = "elliotamcbride"
apiKey = apiKey
fxmlUrl = "https://flightxml.flightaware.com/json/FlightXML2/"
endpoint = 'AirlineFlightSchedules'

#documenation needs the following variables for the payload
#explaination needed for epoch days

today = date.today()
str_today = str(today)
pattern = '%Y-%m-%d'
epoch_today = int(time.mktime(time.strptime(str_today, pattern)))

tomorrow = datetime.date.today() + datetime.timedelta(days = 1)
str_tomorrow = str(tomorrow)
pattern = '%Y-%m-%d'
epoch_tomorrow = int(time.mktime(time.strptime(str_tomorrow, pattern)))

#I have 2 airlines I am interested in as they leave from t5
# I also need anything arriving to heathrow or leaving from heathrow as the payload item won't allow 2 fields

LeavingOrArriving = ['origin', 'destination']
airlines = ['BAW', 'IBE']

schedule_payload_list = []

x = 0 # to log 

for LoA in LeavingOrArriving:
    for i in airlines:
        payload = {'startDate': 1603917334, 'endDate': 1603929334, LoA: 'EGLL', 'airline': i, 'howMany': 6}
        response = requests.get(fxmlUrl + endpoint, params=payload, auth=(username, apiKey))
    if response.status_code == 200:
        schedule_payload_list.append(response.json())
        x += 1
        print("Request is good and " + str(LoA) + " is stored in object " + str(x) + " of schedule_payload_df")
    else:
        print("Error executing request")


Request is good and origin is stored in object 1 of schedule_payload_df
Request is good and destination is stored in object 2 of schedule_payload_df


### Turn 2 API endpoint payloads into one dataframe

Flights are in 2 distinct categories either origin is EGLL (Heathrow) or destination is EGLL so the query and merge has worked. 

Now I can wrangle once rather than twice.

In [107]:
DictionaryOfDataFrames = {} 

for i in range(2):
    DictionaryOfDataFrames["group" + str(i)] = json_normalize(schedule_payload_list[i]['AirlineFlightSchedulesResult']['data'])

flight_schedule_dataframe = pd.concat(DictionaryOfDataFrames.values(), ignore_index=True)

flight_schedule_dataframe

,ident,actual_ident,departuretime,arrivaltime,origin,destination,aircrafttype,meal_service,seats_cabin_first,seats_cabin_business,seats_cabin_coach
0,IBE7345,BAW105,1603917600,1603943100,EGLL,OMDB,B789,First: No meal / Business: No meal / Economy: ...,8,42,166
1,IBE7319,BAW57,1603919400,1603959300,EGLL,FAOR,,First: No meal / Business: No meal / Economy: ...,12,48,159
2,IBE7421,BAW247,1603919700,1603962300,EGLL,SBGR,,First: No meal / Business: No meal / Economy: ...,14,56,227
3,IBE7321,BAW59,1603920600,1603962900,EGLL,FACT,,First: No meal / Business: No meal / Economy: ...,14,56,227
4,IBE7370,BAW157,1603922100,1603944300,EGLL,OKBK,,First: No meal / Business: No meal / Economy: ...,8,49,178
5,IBE7389,BAW194,1603922400,1603954500,KIAH,EGLL,,First: No meal / Business: No meal / Economy: ...,8,49,178
6,IBE4188,AAL46,1603926300,1603954200,KORD,EGLL,B77W,First: No meal / Business: No meal / Economy: ...,8,52,244
7,IBE7450,BAW294,1603926600,1603954200,KORD,EGLL,B789,First: No meal / Business: No meal / Economy: ...,8,42,166
8,IBE7396,BAW206,1603923300,1603953300,KMIA,EGLL,A388,First: No meal / Business: No meal / Economy: ...,10,76,439
9,IBE4192,AAL50,1603928100,1603960800,KDFW,EGLL,B77W,First: No meal / Business: No meal / Economy: ...,8,52,244


### Add Arrival or Departure 
if Origin is EGLL (Heathrow) then it is a Departure flight and we will need to adjust the times we filter on it accordingly.
If it's anything else it is an arrival 

In [108]:
#function to make a new variable to filter on
DepartureOrArrival = flight_schedule_dataframe.origin.apply(lambda x : 'Departing' if x == 'EGLL' else 'Arrival')

flight_schedule_dataframe['DepartureOrArrival'] = DepartureOrArrival

flight_schedule_dataframe.head(20)

,ident,actual_ident,departuretime,arrivaltime,origin,destination,aircrafttype,meal_service,seats_cabin_first,seats_cabin_business,seats_cabin_coach,DepartureOrArrival
0,IBE7345,BAW105,1603917600,1603943100,EGLL,OMDB,B789,First: No meal / Business: No meal / Economy: ...,8,42,166,Departing
1,IBE7319,BAW57,1603919400,1603959300,EGLL,FAOR,,First: No meal / Business: No meal / Economy: ...,12,48,159,Departing
2,IBE7421,BAW247,1603919700,1603962300,EGLL,SBGR,,First: No meal / Business: No meal / Economy: ...,14,56,227,Departing
3,IBE7321,BAW59,1603920600,1603962900,EGLL,FACT,,First: No meal / Business: No meal / Economy: ...,14,56,227,Departing
4,IBE7370,BAW157,1603922100,1603944300,EGLL,OKBK,,First: No meal / Business: No meal / Economy: ...,8,49,178,Departing
5,IBE7389,BAW194,1603922400,1603954500,KIAH,EGLL,,First: No meal / Business: No meal / Economy: ...,8,49,178,Arrival
6,IBE4188,AAL46,1603926300,1603954200,KORD,EGLL,B77W,First: No meal / Business: No meal / Economy: ...,8,52,244,Arrival
7,IBE7450,BAW294,1603926600,1603954200,KORD,EGLL,B789,First: No meal / Business: No meal / Economy: ...,8,42,166,Arrival
8,IBE7396,BAW206,1603923300,1603953300,KMIA,EGLL,A388,First: No meal / Business: No meal / Economy: ...,10,76,439,Arrival
9,IBE4192,AAL50,1603928100,1603960800,KDFW,EGLL,B77W,First: No meal / Business: No meal / Economy: ...,8,52,244,Arrival


### notes so far: 
1. seats in these planes are inconsistant some are missing and there is no documentation on whether they are occupied or not so I logged a support ticket. 
2. Aircraft type is also missing in some instances...All aircrafts have an aircraft type. I may be able to join the actul_ident field on another dataset. 

In [125]:
flight_schedule_dataframe['departuretime'] = pd.to_datetime(flight_schedule_dataframe['departuretime'],unit='s')

flight_schedule_dataframe = flight_schedule_dataframe.drop(flight_schedule_dataframe.columns[[0, 3]], axis=1)  

print(flight_schedule_dataframe.head(2))

KeyError: 'departuretime'

#### filter flights with valid times

In [110]:
today = date.today()

timenow = datetime.datetime.now()

#stringoftimernow = timenow.strftime("%H:%M:%S")

stringoftimernow = '18:26:33'


print (str(today))
print(timenow)
print(stringoftimernow)

2020-10-30
2020-10-30 09:35:48.236165
18:26:33


In [111]:
#filter
#create time filter values

#for departure time filter
timeplus90 = datetime.datetime.now() + timedelta(hours = 1.5)
# for arrival time filter
timeminus30 = datetime.datetime.now() + timedelta(hours = -0.5)

strtimePlus90 = timeplus90.strftime("%H:%M:%S")
strtimeMinus30 = timeminus30.strftime("%H:%M:%S")

#filter out non-relevant departures (greater than time now and smaller than strtimePlus90)
departures_in_next_nintey = flight_schedule_dataframe[(flight_schedule_dataframe.departuretime >= timenow) & (flight_schedule_dataframe.departuretime <= strtimePlus90)]

#filter our non-relevant arrivals (smaller than time now and greater than strtimeMinus30)
filtered_flights_for_valid_time = departures_in_next_nintey[(departures_in_next_nintey.departuretime <= timenow) & (departures_in_next_nintey.departuretime >= strtimePlus90)]

valid_flights = filtered_flights_for_valid_time

print(valid_flights)
type(valid_flights)

Empty DataFrame
Columns: [departuretime, origin, destination, aircrafttype, meal_service, seats_cabin_first, seats_cabin_business, seats_cabin_coach, DepartureOrArrival]
Index: []


pandas.core.frame.DataFrame

### We have our list as per our definition
We now need to make a decision on plane capacities and how to define them

First let's look at the ones simply given by the API - we don't care about class, all seats = 1 person.

#### Extract subtables
Extracting subtables involves the features which are indexed based on their first, second, third, and so on occurence of the feature - First, Business and Economy. These features are grouped together and converting the index appearing in the feature name into a row index.

In short, we're converting from wide to long format and using their idx so that we can add them.


In [119]:
#columns = valid_flights.loc[valid_flights.eval('category == seats'), "field_name"]

# valid_flights['seats_cabin_first'].astype(int) #, 'seats_cabin_business', 'seats_cabin_coach']

# valid_flights['seats_cabin_first']

# columns = ['seats_cabin_first']

# columns_list = valid_flights.columns[valid_flights.columns.str.contains(pat = 'seat')]

# print(columns_list)

# columns = valid_flights.loc[:,columns].head()

data_convictions = extract_subtable(flight_schedule_dataframe, columns, label="seat")

# data_convictions.head()

ValueError: cannot convert float NaN to integer

In [122]:
subdf = flight_schedule_dataframe[['actual_ident','seats_cabin_first','seats_cabin_business','seats_cabin_coach']]
subdf

KeyError: "['actual_ident'] not in index"

In [123]:
stacked = subdf.stack()
stacked

0   seats_cabin_first         8
    seats_cabin_business     42
    seats_cabin_coach       166
1   seats_cabin_first        12
    seats_cabin_business     48
    seats_cabin_coach       159
2   seats_cabin_first        14
    seats_cabin_business     56
    seats_cabin_coach       227
3   seats_cabin_first        14
    seats_cabin_business     56
    seats_cabin_coach       227
4   seats_cabin_first         8
    seats_cabin_business     49
    seats_cabin_coach       178
5   seats_cabin_first         8
    seats_cabin_business     49
    seats_cabin_coach       178
6   seats_cabin_first         8
    seats_cabin_business     52
    seats_cabin_coach       244
7   seats_cabin_first         8
    seats_cabin_business     42
    seats_cabin_coach       166
8   seats_cabin_first        10
    seats_cabin_business     76
    seats_cabin_coach       439
9   seats_cabin_first         8
    seats_cabin_business     52
    seats_cabin_coach       244
10  seats_cabin_first         0
    seat